## Data Importation

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pitching = pd.read_csv("Data/Pitching.csv")
batting = pd.read_csv("Data/Batting.csv")
hof = pd.read_csv("Data/HallOfFame.csv")

In [5]:
batting.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [6]:
pitching.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')

In [7]:
hof.columns

Index(['playerID', 'yearid', 'votedBy', 'ballots', 'needed', 'votes',
       'inducted', 'category', 'needed_note'],
      dtype='object')

In [8]:
for col in ["H", "BB", "IBB", "HBP", "AB", "SH", "SF"]:
    batting[col] = batting[col].fillna(0)

batting["AVG"] = (batting["H"]) / (batting["AB"])

batting["OBP"] = (batting["H"] + batting["BB"] + batting["IBB"] + batting["HBP"]) / (batting["AB"] + batting["BB"] + batting["IBB"] + batting["HBP"] + batting["SH"] + batting["SF"])

In [9]:
batting

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,CS,BB,SO,IBB,HBP,SH,SF,GIDP,AVG,OBP
0,abercda01,1871,1,TRO,NaN,1,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,0.000000
1,addybo01,1871,1,RC1,NaN,25,118.0,30.0,32.0,6.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,NaN,0.271186,0.295082
2,allisar01,1871,1,CL1,NaN,29,137.0,28.0,40.0,4.0,...,1.0,2.0,5.0,0.0,0.0,0.0,0.0,NaN,0.291971,0.302158
3,allisdo01,1871,1,WS3,NaN,27,133.0,28.0,44.0,10.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,0.330827,0.330827
4,ansonca01,1871,1,RC1,NaN,25,120.0,29.0,39.0,11.0,...,2.0,2.0,1.0,0.0,0.0,0.0,0.0,NaN,0.325000,0.336066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101327,zitoba01,2015,1,OAK,AL,3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
101328,zobribe01,2015,1,OAK,AL,67,235.0,39.0,63.0,20.0,...,1.0,33.0,26.0,2.0,0.0,0.0,3.0,5.0,0.268085,0.358974
101329,zobribe01,2015,2,KCA,AL,59,232.0,37.0,66.0,16.0,...,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0,0.284483,0.366038
101330,zuninmi01,2015,1,SEA,AL,112,350.0,28.0,61.0,11.0,...,1.0,21.0,132.0,0.0,5.0,8.0,2.0,6.0,0.174286,0.225389


In [10]:
hof

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,NaN
1,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
2,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
3,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,NaN
4,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,NaN
...,...,...,...,...,...,...,...,...,...
4115,grudzma01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN
4116,ausmubr01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN
4117,hamptmi01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN
4118,castilu01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN


In [11]:
mask = hof["category"] == "Player"
hof = hof[mask]
hof

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,NaN
1,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
2,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
3,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,NaN
4,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,NaN
...,...,...,...,...,...,...,...,...,...
4115,grudzma01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN
4116,ausmubr01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN
4117,hamptmi01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN
4118,castilu01,2016,BBWAA,440.0,330.0,0.0,N,Player,NaN


In [18]:
batting_sum = batting.groupby("playerID", as_index=False).agg({
    "yearID": "count",
    "stint" : "count", 
    "G" : "sum", 
    "AB" : "sum", 
    "R" : "sum",
    "H" : "sum",  
    "2B" : "sum", 
    "3B" : "sum", 
    "HR" : "sum", 
    "RBI" : "sum", 
    "SB" : "sum", 
    "CS" : "sum", 
    "BB" : "sum", 
    "SO" : "sum", 
    "IBB" : "sum", 
    "HBP" : "sum", 
    "SH" : "sum",
    "SF" : "sum", 
    "GIDP" : "sum",
    "OBP" : "mean",
    "AVG" : "mean",
})
batter_hof = pd.merge(batting_sum, hof, on="playerID", how="outer")
batter_hof

,playerID,yearID,stint,G,AB,R,H,2B,3B,HR,...,OBP,AVG,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,aardsda01,9.0,9.0,331.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aaronha01,23.0,23.0,3298.0,12364.0,2174.0,3771.0,624.0,98.0,755.0,...,0.383617,0.301075,1982.0,BBWAA,415.0,312.0,406.0,Y,Player,NaN
2,aaronto01,7.0,7.0,437.0,944.0,102.0,216.0,42.0,6.0,13.0,...,0.276791,0.220660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aasedo01,13.0,13.0,448.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abadan01,3.0,3.0,15.0,21.0,1.0,2.0,0.0,0.0,0.0,...,0.203509,0.039216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,zupofr01,3.0,3.0,16.0,18.0,3.0,3.0,1.0,0.0,0.0,...,0.251282,0.194444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21490,zuvelpa01,9.0,9.0,209.0,491.0,41.0,109.0,17.0,2.0,2.0,...,0.228394,0.152383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21491,zuverge01,10.0,10.0,266.0,142.0,5.0,21.0,2.0,1.0,0.0,...,0.314034,0.187528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21492,zwilldu01,4.0,4.0,366.0,1280.0,167.0,364.0,76.0,15.0,30.0,...,0.290718,0.224028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
batter_hof["AVG"] = (batter_hof["H"]) / (batter_hof["AB"])

batter_hof["OBP"] = (batter_hof["H"] + batter_hof["BB"] + batter_hof["IBB"] + batter_hof["HBP"]) / (batter_hof["AB"] + batter_hof["BB"] + batter_hof["IBB"] + batter_hof["HBP"] + batter_hof["SH"] + batter_hof["SF"])

In [21]:
pitcher_sum = pitching.groupby("playerID", as_index=False).agg({
    "yearID": "count",
    "stint" : "count", 
    "W" : "sum", 
    "L" : "sum", 
    "G" : "sum",
    "GS" : "sum",  
    "CG" : "sum", 
    "SHO" : "sum", 
    "SV" : "sum", 
    "IPouts" : "sum", 
    "H" : "sum", 
    "ER" : "sum", 
    "HR" : "sum", 
    "BB" : "sum", 
    "SO" : "sum", 
    "BAOpp" : "mean", 
    "ERA" : "mean",
    "IBB" : "sum", 
    "WP" : "sum",
    "HBP" : "sum",
    "BK" : "sum",
    "BFP" : "sum", 
    "GF" : "sum",
    "R" : "sum",
    "SH" : "sum",
    "SF" : "sum",
    "GIDP" : "sum"
})
pitcher_hof = pd.merge(pitcher_sum, hof, on="playerID", how="outer")
pitcher_hof

,playerID,yearID,stint,W,L,G,GS,CG,SHO,SV,...,SF,GIDP,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,aardsda01,9.0,9.0,16.0,18.0,331.0,0.0,0.0,0.0,69.0,...,11.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aaronha01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1982.0,BBWAA,415.0,312.0,406.0,Y,Player,NaN
2,aasedo01,13.0,13.0,66.0,60.0,448.0,91.0,22.0,5.0,82.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abadfe01,6.0,6.0,5.0,20.0,258.0,6.0,0.0,0.0,0.0,...,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abbeybe01,6.0,6.0,22.0,40.0,79.0,66.0,52.0,0.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12635,zoldasa01,10.0,10.0,43.0,53.0,250.0,93.0,30.0,5.0,8.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12636,zuberbi01,12.0,12.0,43.0,42.0,224.0,65.0,23.0,3.0,6.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12637,zumayjo01,5.0,5.0,13.0,12.0,171.0,0.0,0.0,0.0,5.0,...,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12638,zuverge01,10.0,10.0,32.0,36.0,265.0,31.0,9.0,2.0,40.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
pitcher_hof["BAOpp"] = pitcher_hof["H"]/(pitcher_hof["BFP"] - pitcher_hof["BB"] - pitcher_hof["IBB"] - pitcher_hof["HBP"] - pitcher_hof["SH"] - pitcher_hof["SF"])
pitcher_hof["ERA"] = pitcher_hof["ER"]/(pitcher_hof["IPouts"]/27)

In [33]:
batter_hof["category"] = batter_hof["category"].fillna("Player")
batter_hof["inducted"] = batter_hof["inducted"].fillna("N")
batter_hof["ballots"] = batter_hof["ballots"].fillna(0)
batter_hof["votes"] = batter_hof["votes"].fillna(0)
batter_hof

,playerID,yearID,stint,G,AB,R,H,2B,3B,HR,...,OBP,AVG,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,aardsda01,9.0,9.0,331.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
1,aaronha01,23.0,23.0,3298.0,12364.0,2174.0,3771.0,624.0,98.0,755.0,...,0.386285,0.304998,1982.0,BBWAA,415.0,312.0,406.0,Y,Player,NaN
2,aaronto01,7.0,7.0,437.0,944.0,102.0,216.0,42.0,6.0,13.0,...,0.291031,0.228814,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
3,aasedo01,13.0,13.0,448.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
4,abadan01,3.0,3.0,15.0,21.0,1.0,2.0,0.0,0.0,0.0,...,0.240000,0.095238,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,zupofr01,3.0,3.0,16.0,18.0,3.0,3.0,1.0,0.0,0.0,...,0.250000,0.166667,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
21490,zuvelpa01,9.0,9.0,209.0,491.0,41.0,109.0,17.0,2.0,2.0,...,0.267399,0.221996,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
21491,zuverge01,10.0,10.0,266.0,142.0,5.0,21.0,2.0,1.0,0.0,...,0.179641,0.147887,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
21492,zwilldu01,4.0,4.0,366.0,1280.0,167.0,364.0,76.0,15.0,30.0,...,0.343728,0.284375,NaN,NaN,0.0,NaN,0.0,N,Player,NaN


In [34]:
pitcher_hof["category"] = pitcher_hof["category"].fillna("Player")
pitcher_hof["inducted"] = pitcher_hof["inducted"].fillna("N")
pitcher_hof["ballots"] = pitcher_hof["ballots"].fillna(0)
pitcher_hof["votes"] = pitcher_hof["votes"].fillna(0)
pitcher_hof

,playerID,yearID,stint,W,L,G,GS,CG,SHO,SV,...,SF,GIDP,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,aardsda01,9.0,9.0,16.0,18.0,331.0,0.0,0.0,0.0,69.0,...,11.0,0.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
1,aaronha01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1982.0,BBWAA,415.0,312.0,406.0,Y,Player,NaN
2,aasedo01,13.0,13.0,66.0,60.0,448.0,91.0,22.0,5.0,82.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
3,abadfe01,6.0,6.0,5.0,20.0,258.0,6.0,0.0,0.0,0.0,...,9.0,6.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
4,abbeybe01,6.0,6.0,22.0,40.0,79.0,66.0,52.0,0.0,1.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12635,zoldasa01,10.0,10.0,43.0,53.0,250.0,93.0,30.0,5.0,8.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
12636,zuberbi01,12.0,12.0,43.0,42.0,224.0,65.0,23.0,3.0,6.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
12637,zumayjo01,5.0,5.0,13.0,12.0,171.0,0.0,0.0,0.0,5.0,...,10.0,0.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN
12638,zuverge01,10.0,10.0,32.0,36.0,265.0,31.0,9.0,2.0,40.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,N,Player,NaN


In [37]:
batter_hof.to_csv("Data/batter_hof.csv", index=False)
pitcher_hof.to_csv("Data/pitcher_hof.csv", index=False)